### Add individual entries to the DB

First, add the entries from another DB

In [19]:
from ase.db import connect
from pathlib import Path
import shutil
from herculestools.dft import RunConfiguration as RC

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.
struct_dir = RC.structures_dir
config_home = RC.home
home = Path.cwd()
copy_db_path = struct_dir/'hexag_perovs_strained_copy.db'
old_db_path = struct_dir/'hexag_perovs_strained.db'
#old_db_path = 'structures/hexag_perovs_wdiscards.db'

db_path = shutil.copy(old_db_path, copy_db_path)
#copy_db_path =  'structures/hexag_perovs_strained_copy.db'

#files_folder = Path('NEB/Ba7Nb4MoO20_p1_e0/').resolve().as_posix()
#name = 'Ba7Nb4MoO20_p1_neb'

with connect(old_db_path) as old_db:
    rows = old_db.select(mask='uniaxial')
    for sys_row in rows:
        row_id = sys_row.id
        sys_atoms = sys_row.toatoms()
        sys_kvps = sys_row.key_value_pairs
        sys_kvps['mask'] = 'x-axis'
        sys_kvps['name'] = sys_kvps['name'].replace('uniaxial', sys_kvps['mask'])
        
        row_dir = Path(sys_row.dir).resolve()
        # Check if the directory exists in the scratch
        if Path(scratch_dir := row_dir.as_posix().replace('energy','scratch3')).is_dir():
            #print(f"The directory {scratch_dir} exists")
            old_direc = scratch_dir
        elif Path(home_dir := row_dir.as_posix().replace('scratch3','energy')).is_dir():
            # Try to fetch the directory in the home
            #print(f"The directory {home_dir} exists")
            old_direc = home_dir
        else:
            print(f"No directory found for {row_dir}")
        
        rename_dir = old_direc.replace('uniaxial', sys_kvps['mask'])
        print(f"Renaming {old_direc} to {rename_dir}")
        # Rename the old directories to the new ones
        shutil.move(old_direc, rename_dir)
        # try:
        #     shutil.move(old_direc, new_dir)
        # except FileNotFoundError:
        #     # Check if the directory was already renamed
        #     if Path(new_dir).exists():
        #         print(f"Directory {new_dir} already exists")
        #     else:
        #         print(f"Directory {old_direc} not found")
        #         continue
        try:
            new_dir = Path(rename_dir).relative_to(home).as_posix()
        except ValueError:
            new_dir = Path(rename_dir).relative_to(config_home).as_posix()

        sys_kvps['dir'] = new_dir
        
        with connect(copy_db_path) as new_db:
           new_db.update(id=row_id, name=sys_kvps['name'], mask=sys_kvps['mask'], dir=sys_kvps['dir'])    
        #break

Renaming /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/uniaxial/e0 to /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/x-axis/e0
Renaming /home/scratch3/armoma/phd/hex_perovs/strained/preNEB/Ba7Nb4MoO20_p1_uniaxial_e0/init to /home/scratch3/armoma/phd/hex_perovs/strained/preNEB/Ba7Nb4MoO20_p1_x-axis_e0/init
Renaming /home/scratch3/armoma/phd/hex_perovs/strained/preNEB/Ba7Nb4MoO20_p1_uniaxial_e0/final to /home/scratch3/armoma/phd/hex_perovs/strained/preNEB/Ba7Nb4MoO20_p1_x-axis_e0/final
Renaming /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/uniaxial/s1 to /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/x-axis/s1
Renaming /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/uniaxial/s3 to /home/scratch3/armoma/phd/hex_perovs/strained/distorted/Ba7Nb4MoO20_p1/x-axis/s3
Renaming /home/scratch3/armoma/phd/hex_perovs/strained/preNEB/Ba7Nb4MoO20_p1_uniaxial_s1/init to /home/scratch

Then, modify the entry to match as best as possible with respect to the currrent scheme.

In [2]:

with connect(new_db_path) as db:
    sys_row = db.get(name=name)
    sys_id = sys_row.id
    sys_atoms = sys_row.toatoms()
    kvps = [kvp for kvp in sys_row]

    keys_values = {}
    keys_values['name'] = "Ba7Nb4MoO20_p1_e0_neb"
    keys_values['dir'] = files_folder
    keys_values['dopant']=int(sys_row.name[-5])
    keys_values['in_plane'] = 1.00
    db.update(sys_id, **keys_values)



Change the values of several entries at a time.

#### NEBs

In [25]:
from ase.db import connect
from pathlib import Path
import shutil

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.

db_path = 'structures/hexag_perovs_strained.db'
#copy_path = 'structures/hexag_perovs_strained_copy.db'

#copy_path = shutil.copy(db_path, copy_path)

#with connect(copy_path) as db:
with connect(db_path) as db:
    get_rows = db.select('barrier')
    kvps = {}
    for row in get_rows:
        row_id = row.id
        kvps['delta_e'] = abs(row.delta_e)
        name_parts = row.name.split('_')
        dist_name = name_parts[-2]
        print(dist_name)
        kvps['dopant'] = int(name_parts[1][-1])
        if dist_name=='s1':
            kvps['in_plane'] = 1.015
        elif dist_name=='s3':
            kvps['in_plane']=1.030
        elif dist_name=='c1':
            kvps['in_plane']=0.985
        elif dist_name=='c3':
            kvps['in_plane']=0.970
        elif dist_name=='e0':
            kvps['in_plane']=1.000
        
        db.update(row_id, **kvps)
        print(row_id,kvps)
        #break


s1
28 {'delta_e': 0.6102335000000494, 'dopant': 1, 'in_plane': 1.015}
s3
31 {'delta_e': 0.6151655599999231, 'dopant': 1, 'in_plane': 1.03}
s1
32 {'delta_e': 0.6243580099999235, 'dopant': 1, 'in_plane': 1.015}
e0
33 {'delta_e': 0.5970954699998856, 'dopant': 1, 'in_plane': 1.0}


#### Others

In [2]:
from ase.db import connect
from pathlib import Path
import shutil

# Read and generate an atoms entry from the NEB run. Write/Update it to the DB.

db_path = 'structures/hexag_perovs_strained.db'
#copy_path = 'structures/hexag_perovs_strained_copy.db'

#copy_path = shutil.copy(db_path, copy_path)

#with connect(copy_path) as db:
with connect(db_path) as db:
    get_rows = db.select(selection='in_plane=1.010')
    kvps = {}
    for row in get_rows:
        row_id = row.id
        row_in_plane = row.in_plane
        if row_in_plane==1.010:
            in_plane = 1.015
        elif row_in_plane==0.990:
            in_plane = 0.985

        print(row.id, row_in_plane)
        #db.update(row_id, in_plane=in_plane)
        #break
        

In [36]:
print([row for row in get_rows])

[]


### Change the resources of the queued jobs to the epyc96 partition.

In [1]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources

# Get the entries from the active pq database and modify their resources
with PersistentQueue() as pq:
    entries = pq.get_entries()
    
s = Selection(states='q')
targets = s.filter(entries)

# Modify the resources of the entries
new_res = Resources.from_string("96:1:epyc96:50h")
with PersistentQueue() as pq:
    for en in targets:
        en._task.resources = new_res
        pq.save_resources(en.key, new_res, False)
        print(en)
    
        
    # pq_key = en.key
    # #print(en)
    # with PersistentQueue() as pq:
    #     if pq.get_code(pq_key).name != 'neb.py':
    #         print(f"New resources: {new_res1} for pq key: {pq_key}")
    #         pq.save_resources(pq_key, Resources(new_res1), all=False)
    #         print(en)
        #else:
        #    print(f"New resources: {new_res2} for pq key: {pq_key}")
        #    pq.save_resources(pq_key, Resources(new_res2), all=True)
        
        #pq.resubmit(en)
    
    #break

Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -2.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=1, name='apply_strain.py', mq_id=7524397, state='q', err_txt=None, entry_group_key=1, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': -1.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=4, name='apply_strain.py', mq_id=7524398, state='q', err_txt=None, entry_group_key=2, pq_args=None, data=None)
Entry(task=Task(code=PosixPath('/home/energy/armoma/phd/hex_perovs/strained/codes/apply_strain.py'), args={'system_id': 1801, 'name': 'Ba7Nb4MoO20_p1', 'in_plane': 0.0}, resources=Resources(cores=96, processes=1, tmax=180000, nodename='epyc96')), entry_key=7, na

### Modify the state of an entry

In [1]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources
from perqueue.task_classes.util_classes import EntryState
import perqueue.coordinator as coord

# Get the entries from the active pq database and modify their resources
with PersistentQueue() as pq:
    entries = pq.get_entries()
    
s = Selection(ids=[54,32])
targets = s.filter(entries)

#new_state = EntryState('q')
with PersistentQueue() as pq:
    for en in targets:
        # Change the state of the entry
        coord.set_state(pq._backend, en.key, EntryState.succeeded, None)
        
        # Populate the data field in the entry
        #coord.set_data(pq._backend, en.key, {'db_id': 1})
        
        # Resubmit the entry
        #pq.resubmit(en)
        
        #break
        

#### Change the args of an entry.

In [3]:
from perqueue.queue import PersistentQueue
from perqueue.selection import Selection
from ase.db import connect
from perqueue.task_classes.util_classes import Resources
from perqueue.task_classes.util_classes import EntryState
import perqueue.coordinator as coord

# Get the succesful NEB jobs with PQ
with PersistentQueue() as pq:
    entries = pq.get_entries()

s = Selection(ids=32)
targets = s.filter(entries=entries)

#args = {'climb': True}
data = {'trajectory': 'Ba7Nb4MoO20_p1_biaxial_e0_1.traj', 'neb_id': 43}
with PersistentQueue() as pq:
    for en in targets:
        # Store the data in the entry
        coord.set_data(pq._backend, en.key, data)
        
        #key = en.key
        #pq.save_args(key, args, False)
        #pq.resubmit(en)

In [11]:
resources = Resources.from_string('48:1:xeon24el8_test:10m')
with PersistentQueue() as pq:
    entry= pq.get_entry(key=12)
    pq.save_resources(entry.key, resources, False)

In [ ]:
data = {'trajectory': 'Ba7Nb4MoO20_p1_biaxial_e0_1.traj', 'neb_id': 43}